# Analyzing results for Spatial Navigation

## Imports and configuration

In [ ]:
import json
import matplotlib as mpl
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import numpy as np
import os
import pathlib
import scipy.stats as stats
from sklearn.decomposition import PCA
import sys
import torch

sys.path.append("..")

from models import rnn
from tasks import spatial_navigation
import utils

In [ ]:
if "Arial" not in fm.get_font_names():
    font_path = pathlib.Path.home() / "fonts" / "arial.ttf"  # Set to correct path
    fm.fontManager.addfont(font_path)

plt.rcParams["font.family"] = "Arial"
plt.rcParams["font.size"] = 10
plt.rcParams["axes.linewidth"] = 1.2
plt.rcParams["xtick.major.width"] = 1.2
plt.rcParams["ytick.major.width"] = 1.2

In [ ]:
active = "#2ca7c5"
quiescent = "#ee3233"

active_colors = [
    "#ffffff",
    "#d4edf3",
    "#aadbe7",
    "#80cadc",
    "#56b8d0",
    "#2ca7c5",
    "#2796b1",
    "#1e7489",
    "#165362",
    "#0d323b",
]
lcmap_active = mpl.colors.LinearSegmentedColormap.from_list("lcmap_active", active_colors)
norm = mpl.colors.Normalize(vmin=0, vmax=1.5)
lcmap_active = mpl.cm.ScalarMappable(norm=norm, cmap=lcmap_active)
lcmap_active.set_array([])

quiescent_colors = [
    "#ffffff",
    "#fbd6d6",
    "#f8adad",
    "#f48484",
    "#f15a5b",
    "#ee3233",
    "#d62d2d",
    "#a62323",
    "#771919",
    "#470f0f",
]
lcmap_quiescent = mpl.colors.LinearSegmentedColormap.from_list("lcmap_quiescent", quiescent_colors)
norm = mpl.colors.Normalize(vmin=0, vmax=1.5)
lcmap_quiescent = mpl.cm.ScalarMappable(norm=norm, cmap=lcmap_quiescent)
lcmap_quiescent.set_array([])

## Load results

In [ ]:
results = pathlib.Path("../results")
noise = 0.0001
config_name = f"noisy_unbiased_{noise}"
seed_dirs = [results / "spatial_navigation" / f"{config_name}_{seed}" for seed in range(5)]
figures = results / "spatial_navigation" / "figures"
pathlib.Path.mkdir(figures, exist_ok=True)

## Visualizations 

### Loss

In [ ]:
test_metrics = [json.load(open(d / "test_metrics.json", "r")) for d in seed_dirs]
start, end, skip = 100, 2501, 100
test_losses = np.array([[f[str(i)]["loss"] for i in range(start, end, skip)] for f in test_metrics])
test_posmse = np.array([[f[str(i)]["pos_mse"] for i in range(start, end, skip)] for f in test_metrics])

In [ ]:
losses_mean, losses_std = test_losses.mean(axis=0), test_losses.std(axis=0)
posmse_mean, posmse_std = test_posmse.mean(axis=0), test_posmse.std(axis=0)

In [ ]:
fig, ax = plt.subplots(figsize=(1.5, 1.5))

ax.spines[["right", "top"]].set_visible(False)
ax.set_box_aspect(1)
ax.errorbar(np.arange(start, end, skip), losses_mean, yerr=losses_std, fmt="-", c="gray")
ax.set_xlabel("batches")
ax.set_ylabel("test loss", color="gray")
ax.set_yticks(np.linspace(1e-5, 5e-5, 5))
ax.tick_params(axis="both", which="major", labelsize=8)
ax.tick_params(axis="both", which="minor", labelsize=8)
ax.yaxis.get_offset_text().set_fontsize(8)

ax2 = ax.twinx()
ax2.spines[["left", "top"]].set_visible(False)
ax2.set_box_aspect(1)
ax2.errorbar(np.arange(start, end, skip), posmse_mean, yerr=posmse_std, fmt="-", c="black")
ax2.set_ylabel("position decoding error", color="black")
ax2.tick_params(axis="both", which="major", labelsize=8)
ax2.tick_params(axis="both", which="minor", labelsize=8)

plt.show()

In [ ]:
fig.savefig(figures / f"{config_name}-loss.pdf", bbox_inches="tight", pad_inches=0)

### Population activity dimensionality (PCs vs cumulative explained variance)

In [ ]:
ev_active = []
ev_quiescent = []

In [ ]:
for seed in range(5):
    utils.set_random_seeds(seed)

    epoch = 2500
    model = torch.load(seed_dirs[seed] / f"model_{epoch}.pt", map_location="cpu")
    model.set_device("cpu")
    model.eval()

    task = model.task
    task.batch_size = 1000

    test_data = task.get_test_batch()
    t_quiescent = 200
    quiescent_inputs = torch.zeros(task.batch_size, t_quiescent, 2)

    h_active, _ = model(test_data["data"], test_data["init_state"])
    h_active = h_active.cpu().detach().numpy().reshape(-1, model.n_rec)
    model.sigma_rec *= np.sqrt(2)
    h_quiescent, _ = model(quiescent_inputs, test_data["init_state"])
    h_quiescent = h_quiescent.cpu().detach().numpy().reshape(-1, model.n_rec)

    pca = PCA()
    pca.fit(h_active)
    ev_active.append(pca.explained_variance_ratio_)
    pca.fit(h_quiescent)
    ev_quiescent.append(pca.explained_variance_ratio_)

In [ ]:
ev_active = np.array(ev_active)
ev_quiescent = np.array(ev_quiescent)

In [ ]:
fig, ax = plt.subplots(figsize=(1.5, 1.5))
ax.spines[["right", "top"]].set_visible(False)
ax.set_box_aspect(1)

ax.errorbar(
    np.arange(1, 11),
    ev_active.cumsum(axis=-1).mean(axis=0)[:10],
    yerr=ev_active.cumsum(axis=-1).std(axis=0)[:10],
    fmt="-",
    c=active,
    label="active",
)
ax.errorbar(
    np.arange(1, 11),
    ev_quiescent.cumsum(axis=-1).mean(axis=0)[:10],
    yerr=ev_quiescent.cumsum(axis=-1).std(axis=0)[:10],
    fmt="-",
    c=quiescent,
    label="quiescent",
)

ax.set_xlabel("# of PCs")
ax.set_ylabel("explained variance")
ax.set_xticks(np.arange(1, 11))
ax.set_yticks(np.arange(0.4, 1.0, 0.1))
ax.tick_params(axis="both", which="major", labelsize=8)
ax.tick_params(axis="both", which="minor", labelsize=8)

leg = ax.legend(loc="best", frameon=False)

for handle, text in zip(leg.legend_handles, leg.get_texts()):
    text.set_color(handle.get_color())
    handle.set_visible(False)

plt.show()

In [ ]:
fig.savefig(figures / f"{config_name}-explained_variance.pdf", bbox_inches="tight", pad_inches=0)

### Neural activity PCs

In [ ]:
quiescence = "scaled"
# One of:
#   - "scaled" (quiescent_noise = np.sqrt(2) * active_noise)
#   - "same" (quiescent_noise = active_noise)
#   - "absolute" (quiescent_noise = np.sqrt(2 * noise) where noise is defined earlier e.g. 0.0001

In [ ]:
seed = 0
utils.set_random_seeds(seed)

epoch = 2500
model = torch.load(seed_dirs[seed] / f"model_{epoch}.pt", map_location="cpu")
model.set_device("cpu")
model.eval()

task = model.task
task.batch_size = 200

test_data = task.get_test_batch()
t_quiescent = 1000
quiescent_inputs = torch.zeros(task.batch_size, t_quiescent, 2)

h_active = model(test_data["data"], test_data["init_state"])

if quiescence == "scaled":
    model.sigma_rec *= np.sqrt(2)
elif quiescence == "absolute":
    model.sigma_rec = np.sqrt(2 * noise)
elif quiescence == "same":
    pass

h_quiescent = model(quiescent_inputs, test_data["init_state"])

h_active_ = h_active[0].cpu().detach().numpy().reshape(-1, model.n_rec)
h_quiescent_ = h_quiescent[0].cpu().detach().numpy().reshape(-1, model.n_rec)

pca = PCA()
h_active_pca = pca.fit_transform(h_active_)
h_quiescent_pca = pca.transform(h_quiescent_)

In [ ]:
active_xy = task.place_cells.decode_pos(h_active[1])
active_d = (active_xy**2).sum(axis=-1) ** 0.5
quiescent_xy = task.place_cells.decode_pos(h_quiescent[1])
quiescent_d = (quiescent_xy**2).sum(axis=-1) ** 0.5

In [ ]:
fig, ax = plt.subplots(figsize=(7.5, 1.5))
ax.set_box_aspect(1)

skip_traj = 2
shift_time = 7
skip_time = 20
t_task = task.sequence_length
for i in range(0, task.batch_size, skip_traj):
    plt.scatter(
        h_active_pca[i * t_task + shift_time : (i + 1) * t_task : skip_time, 0],
        h_active_pca[i * t_task + shift_time : (i + 1) * t_task : skip_time, 1],
        c=lcmap_active.to_rgba(active_d[i, shift_time::skip_time]),
    )

ax.grid(visible=False)
ax.set_xlabel("PC-1")
ax.set_ylabel("PC-2")
ax.spines[["right", "top"]].set_visible(False)
ax.tick_params(axis="both", which="major", labelsize=8)
ax.tick_params(axis="both", which="minor", labelsize=8)
ax.set_xticks([-1, 0, 1])
ax.set_yticks([-1, 0, 1])
ax.set_xlim(-1.55, 1.55)
ax.set_ylim(-1.55, 1.55)

cb = plt.colorbar(mappable=lcmap_active, ax=ax)
cb.ax.tick_params(labelsize=8)
cb.set_label(label="distance from center in\n(x, y)-space", labelpad=10, fontsize=8)

In [ ]:
fig.savefig(figures / f"{config_name}_{seed}-pca_active.pdf", bbox_inches="tight", pad_inches=0)

In [ ]:
fig, ax = plt.subplots(figsize=(7.5, 1.5))
ax.set_box_aspect(1)

skip_traj = 2
shift_time = 7
skip_time = 20
for i in range(0, task.batch_size, skip_traj):
    plt.scatter(
        h_quiescent_pca[i * t_quiescent + shift_time : (i + 1) * t_quiescent : skip_time, 0],
        h_quiescent_pca[i * t_quiescent + shift_time : (i + 1) * t_quiescent : skip_time, 1],
        c=lcmap_quiescent.to_rgba(quiescent_d[i, shift_time::skip_time]),
    )

ax.grid(visible=False)
ax.set_xlabel("PC-1")
ax.set_ylabel("PC-2")
ax.spines[["right", "top"]].set_visible(False)
ax.tick_params(axis="both", which="major", labelsize=8)
ax.tick_params(axis="both", which="minor", labelsize=8)
ax.set_xticks([-1, 0, 1])
ax.set_yticks([-1, 0, 1])
ax.set_xlim(-1.55, 1.55)
ax.set_ylim(-1.55, 1.55)

cb = plt.colorbar(mappable=lcmap_quiescent, ax=ax)
cb.ax.tick_params(labelsize=8)
cb.ax.yaxis.set_ticks_position("left")
cb.set_label(label="distance from center in\n(x, y)-space", labelpad=10, fontsize=8)

In [ ]:
fig.savefig(
    figures / f"{config_name}_{seed}-pca_quiescent_{quiescence}.pdf", bbox_inches="tight", pad_inches=0
)

### Decoded outputs

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, layout="compressed", figsize=(3, 1.5))

ax = axes[0]
ax.set_box_aspect(1)
traj_skip = 8

for i in range(0, task.batch_size, traj_skip):
    ax.plot(active_xy[i, :, 0], active_xy[i, :, 1], c=active)
    ax.scatter(active_xy[i, 0, 0], active_xy[i, 0, 1], c="k", label="start", s=10, zorder=3)
    ax.scatter(active_xy[i, -1, 0], active_xy[i, -1, 1], c="k", label="end", marker="^", s=10, zorder=3)

ax.grid(visible=False)
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.tick_params(axis="both", which="major", labelsize=8)
ax.tick_params(axis="both", which="minor", labelsize=8)
ax.set_xlim(-1.15, 1.15)
ax.set_ylim(-1.15, 1.15)
ax.set_xticks([-1, 0, 1])
ax.set_yticks([-1, 0, 1])

ax = axes[1]
ax.set_box_aspect(1)
traj_skip = 4

for i in range(0, task.batch_size, traj_skip):
    ax.plot(quiescent_xy[i, :, 0], quiescent_xy[i, :, 1], c=quiescent)
    ax.scatter(quiescent_xy[i, 0, 0], quiescent_xy[i, 0, 1], c="k", label="start", s=10, zorder=3)
    ax.scatter(quiescent_xy[i, -1, 0], quiescent_xy[i, -1, 1], c="k", label="end", marker="^", s=10, zorder=3)

ax.grid(visible=False)
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.tick_params(axis="both", which="major", labelsize=8)
ax.tick_params(axis="both", which="minor", labelsize=8)
ax.set_xlim(-1.15, 1.15)
ax.set_ylim(-1.15, 1.15)
ax.set_xticks([-1, 0, 1])
ax.set_yticks([-1, 0, 1])

handles, labels = ax.get_legend_handles_labels()
by_label = dict(zip(labels, handles))
fig.legend(
    by_label.values(),
    by_label.keys(),
    frameon=False,
    bbox_to_anchor=(0.49, -0.07),
    loc="lower center",
    borderaxespad=0,
    ncol=2,
    handletextpad=0,
)

In [ ]:
fig.savefig(figures / f"{config_name}_{seed}-behavior-q_{quiescence}.pdf", bbox_inches="tight", pad_inches=0)

### Density plots

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(3, 3))

# Active
X = active_xy.reshape(-1, 2).cpu().detach().numpy()
x, y = X[:, 0], X[:, 1]
xmin, xmax = -1.25, 1.25
ymin, ymax = -1.25, 1.25
xx, yy = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]

positions = np.vstack([xx.ravel(), yy.ravel()])
values = np.vstack([x, y])
kernel = stats.gaussian_kde(values)
f = np.reshape(kernel(positions).T, xx.shape)

ax = axes[0]
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
cfset = ax.contourf(xx, yy, f, cmap="coolwarm", levels=np.linspace(0.0, 1.2, 9))
ax.imshow(np.rot90(f), cmap="coolwarm", extent=[xmin, xmax, ymin, ymax])
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.tick_params(axis="both", which="major", labelsize=8)
ax.tick_params(axis="both", which="minor", labelsize=8)
ax.set_xticks([-1, 0, 1])
ax.set_yticks([-1, 0, 1])

# Quiescent
traj_skip = 5
X = quiescent_xy[::traj_skip, :, :].reshape(-1, 2).cpu().detach().numpy()
x, y = X[:, 0], X[:, 1]
xmin, xmax = -1.25, 1.25
ymin, ymax = -1.25, 1.25
xx, yy = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]

positions = np.vstack([xx.ravel(), yy.ravel()])
values = np.vstack([x, y])
kernel = stats.gaussian_kde(values)
f = np.reshape(kernel(positions).T, xx.shape)

ax = axes[1]
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
cfset = ax.contourf(xx, yy, f, cmap="coolwarm", levels=np.linspace(0.0, 1.2, 9))
ax.imshow(np.rot90(f), cmap="coolwarm", extent=[xmin, xmax, ymin, ymax])
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.tick_params(axis="both", which="major", labelsize=8)
ax.tick_params(axis="both", which="minor", labelsize=8)
ax.set_xticks([-1, 0, 1])
ax.set_yticks([-1, 0, 1])

cb = fig.colorbar(
    cfset, ax=axes.ravel().tolist(), orientation="horizontal", location="bottom", label="density"
)
cb.ax.tick_params(labelsize=8)

In [ ]:
fig.savefig(
    figures / f"{config_name}_{seed}-kde_output-q_{quiescence}.pdf", bbox_inches="tight", pad_inches=0
)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(3, 3))

# Active
X = h_active_pca[:, :2]
x, y = X[:, 0], X[:, 1]
delta_x = (max(x) - min(x)) / 10
delta_y = (max(y) - min(y)) / 10
xmin = min(x) - delta_x
xmax = max(x) + delta_x
ymin = min(y) - delta_y
ymax = max(y) + delta_y
xx, yy = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]

positions = np.vstack([xx.ravel(), yy.ravel()])
values = np.vstack([x, y])
kernel = stats.gaussian_kde(values)
f = np.reshape(kernel(positions).T, xx.shape)

ax = axes[0]
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
cfset = ax.contourf(xx, yy, f, cmap="coolwarm", levels=np.linspace(0.0, 1.6, 9))
ax.imshow(np.rot90(f), cmap="coolwarm", extent=[xmin, xmax, ymin, ymax])
ax.set_xlabel("PC-1")
ax.set_ylabel("PC-2")
ax.tick_params(axis="both", which="major", labelsize=8)
ax.tick_params(axis="both", which="minor", labelsize=8)
ax.set_xticks([-1, 0, 1])
ax.set_yticks([-1, 0, 1])

# Quiescent
traj_skip = 5
X = h_quiescent_pca[::traj_skip, :2]
x, y = X[:, 0], X[:, 1]
xx, yy = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]

positions = np.vstack([xx.ravel(), yy.ravel()])
values = np.vstack([x, y])
kernel = stats.gaussian_kde(values)
f = np.reshape(kernel(positions).T, xx.shape)

ax = axes[1]
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
cfset = ax.contourf(xx, yy, f, cmap="coolwarm", levels=np.linspace(0.0, 1.6, 9))
ax.imshow(np.rot90(f), cmap="coolwarm", extent=[xmin, xmax, ymin, ymax])
ax.set_xlabel("PC-1")
ax.set_ylabel("PC-2")
ax.tick_params(axis="both", which="major", labelsize=8)
ax.tick_params(axis="both", which="minor", labelsize=8)
ax.set_xticks([-1, 0, 1])
ax.set_yticks([-1, 0, 1])

cb = fig.colorbar(
    cfset, ax=axes.ravel().tolist(), orientation="horizontal", location="bottom", label="density"
)
cb.ax.tick_params(labelsize=8)

In [ ]:
fig.savefig(figures / f"{config_name}_{seed}-kde_pca-q_{quiescence}.pdf", bbox_inches="tight", pad_inches=0)

### Average output variance

In [ ]:
variance_files = [
    json.load(open(results / "spatial_navigation" / f"variance_{noise}_{seed}.json", "r"))
    for seed in range(5)
]
variance = np.array([list(f.values()) for f in variance_files])

In [ ]:
fig, ax = plt.subplots(figsize=(3, 1.5))
c = "black"
ax.boxplot(
    variance,
    labels=["noiseless\nunbiased", "noiseless\nbiased", "noisy\nunbiased", "noisy\nbiased"],
    patch_artist=True,
    boxprops=dict(facecolor=quiescent, color=c),
    capprops=dict(color=c, linewidth=0),
    whiskerprops=dict(color=c),
    flierprops=dict(color=c, markeredgecolor=c),
    medianprops=dict(color=c),
)
ax.tick_params(axis="both", which="major", labelsize=8)
ax.tick_params(axis="both", which="minor", labelsize=8)
ax.tick_params(axis="x", bottom=False)
ax.spines[["right", "top"]].set_visible(False)
ax.set_ylabel("avg. variance")

In [ ]:
fig.savefig(figures / f"output_variance_{noise}.pdf", bbox_inches="tight", pad_inches=0)

### KL divergence

In [ ]:
kl_files = [
    json.load(open(results / "spatial_navigation" / f"kl_{noise}_{seed}.json", "r")) for seed in range(5)
]

In [ ]:
def outer_order(seed):
    return [
        f"noisy_unbiased_0.0001_{seed}",
        f"no-noise_unbiased_{seed}",
        "uniform",
        f"noisy_biased_0.0001_{seed}",
        f"no-noise_biased_{seed}",
        "random",
    ]


kl_files = [{k: kl_files[seed][k] for k in outer_order(seed)} for seed in range(5)]

In [ ]:
def inner_order(seed):
    return [
        f"noisy_unbiased_0.0001_{seed}",
        f"no-noise_unbiased_{seed}",
        f"noisy_biased_0.0001_{seed}",
        f"no-noise_biased_{seed}",
    ]


kl_files_n = []
for seed in range(5):
    new_dict = {}
    for k, v in kl_files[seed].items():
        new_dict[k] = {k_: v[k_] for k_ in inner_order(seed)}
    kl_files_n.append(new_dict)

In [ ]:
kl = np.array([[list(f.values()) for f in kl_files_n[seed].values()] for seed in range(5)])

In [ ]:
kl_mean = kl.mean(axis=0)

In [ ]:
fig, ax = plt.subplots(figsize=(3, 1.5), dpi=600)

im = ax.matshow(kl_mean.T, cmap="coolwarm", interpolation="nearest")

ax.set_xticks(
    range(6),
    [
        r"$\mathregular{U}^{\sigma}$",
        "U",
        r"$\mathcal{U}$",
        r"$\mathregular{B}^{\sigma}$",
        "B",
        r"$\mathregular{R}^{\sigma}$",
    ],
)
ax.set_yticks(range(4), [r"$\mathregular{U}^{\sigma}$", "U", r"$\mathregular{B}^{\sigma}$", "B"])
ax.tick_params(axis="both", which="major", labelsize=8)
ax.tick_params(axis="both", which="minor", labelsize=8)
ax.tick_params(axis="y", left=False)
ax.tick_params(axis="x", labeltop=True, labelbottom=False, top=False, bottom=False)
ax.set_xlabel("quiescent")
ax.set_ylabel("active")

cb = fig.colorbar(im, location="right")
cb.set_label("KL divergence")
cb.ax.tick_params(axis="both", which="major", labelsize=8)
cb.ax.set_yticks(np.linspace(kl_mean.min(), kl_mean.max(), 4))

In [ ]:
fig.savefig(figures / f"output_kl_{noise}.pdf", bbox_inches="tight", pad_inches=0)

### Average point-wise distance

In [ ]:
distance_files = [
    json.load(open(results / "spatial_navigation" / f"distance_{noise}_{seed}.json")) for seed in range(5)
]

In [ ]:
unbiased = True
if unbiased:
    un = "un"
else:
    un = ""

distance = [
    [distance_files[seed]["active"][f"noisy_{un}biased_0.0001_{seed}"] for seed in range(5)],
    [distance_files[seed]["quiescent"][f"noisy_{un}biased_0.0001_{seed}"] for seed in range(5)],
    [distance_files[seed]["active"][f"no-noise_{un}biased_{seed}"] for seed in range(5)],
    [distance_files[seed]["quiescent"][f"no-noise_{un}biased_{seed}"] for seed in range(5)],
    [distance_files[seed]["quiescent_noisy"][f"no-noise_{un}biased_{seed}"] for seed in range(5)],
]
distance = [np.array(d).flatten() for d in distance]

In [ ]:
fig, ax = plt.subplots(figsize=(2.7, 1.5))
c = "black"
ax.boxplot(
    distance,
    labels=["NT\nactive", "NT\nquiescent", "DT\nactive", "DT\nquiescent", "DT\nnoisy\nquiescent"],
    patch_artist=True,
    boxprops=dict(facecolor="gray", color=c),
    capprops=dict(color=c, linewidth=0),
    whiskerprops=dict(color=c),
    flierprops=dict(color=c, markeredgecolor=c),
    medianprops=dict(color=c),
)
ax.tick_params(axis="both", which="major", labelsize=8)
ax.tick_params(axis="both", which="minor", labelsize=8)
ax.tick_params(axis="x", bottom=False)
ax.spines[["right", "top"]].set_visible(False)
ax.set_ylabel("avg. distance")

In [ ]:
fig.savefig(figures / f"output_distance_{noise}.pdf", bbox_inches="tight", pad_inches=0)